In [2]:
#lab 7 fuzzy real time

import cv2
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl

# Fuzzy variables
brightness = ctrl.Antecedent(np.arange(256), 'brightness')
edge_intensity = ctrl.Antecedent(np.arange(256), 'edge_intensity')
classification = ctrl.Consequent(np.arange(101), 'classification')

# Define membership functions
def set_mf(var, labels, ranges):
    for label, rng in zip(labels, ranges):
        var[label] = fuzz.trimf(var.universe, rng)

set_mf(brightness, ['dark', 'normal', 'bright'], [(0, 50, 100), (50, 127, 200), (150, 200, 255)])
set_mf(edge_intensity, ['low', 'medium', 'high'], [(0, 50, 100), (50, 127, 200), (150, 200, 255)])
set_mf(classification, ['low', 'medium', 'high'], [(0, 25, 50), (25, 50, 75), (50, 75, 100)])

# Rules
rules = [
    ctrl.Rule(brightness['dark'] | edge_intensity['low'], classification['low']),
    ctrl.Rule(brightness['normal'] | edge_intensity['medium'], classification['medium']),
    ctrl.Rule(brightness['bright'] | edge_intensity['high'], classification['high']),
]

classifier = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))

# Real-time fuzzy classification
def real_time_classification():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret: break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        classifier.input['brightness'] = np.mean(gray)
        classifier.input['edge_intensity'] = np.mean(cv2.Canny(gray, 100, 200))
        classifier.compute()

        cv2.putText(frame, f'Class: {classifier.output["classification"]:.1f}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Fuzzy Classification', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'): break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    real_time_classification()
